In [2]:
import numpy as np

class Dual:
    """ 
    This is a class for mathematical operations on dual numbers. 
      
    Attributes: 
        val (float): function value of dual number. 
        der (float): derivative part of dual number. 
    """
    
    def __init__(self, x, der=1):
        """ 
        The constructor for Dual class. 
  
        Parameters: 
            val (float): function value of dual number. 
            der (float): derivative part of dual number.   
        """
        self.val = x
        self.der = der
        
    ## UNARY OPERATIONS
        
        
    def __neg__(self):
        """ 
        The function for unitary negative operation. 
  
        Parameters: 
            self (Dual): The dual number to perform the unitary negative operation. 
          
        Returns: 
            Dual: A dual number which contains the result. 
        """
        return Dual(-self.val, -self.der)
    
    
    def __pos__(self):
        """ 
        The function for unitary plus operation. 
  
        Parameters: 
            self (Dual): The dual number to perform the unitary plus operation. 
          
        Returns: 
            Dual: A dual number which contains the result. 
        """
        return Dual(+self.val, +self.der)
        
        
    ## PLUS OPERATIONS
    
    def __add__(self, other):
        """ 
        The function to add two dual number or a dual number from the left. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (Dual / float): The dual/float number to be added.
          
        Returns: 
            Dual: A dual number which contains the sum. 
        """
        try:
            return Dual(self.val + other.val, self.der + other.der)
        except AttributeError:
            return Dual(self.val + other, self.der)
    
    def __radd__(self, other):
        """ 
        The function to add a dual number from the right. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (float): The float number to be added.
          
        Returns: 
            Dual: A dual number which contains the sum. 
        """
        return Dual(other + self.val, self.der)
    
    
    ## MINUS OPERATIONS
    
    def __sub__(self, other):
        """ 
        The function to substract two dual number or a dual number from the left. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (Dual / float): The dual/float number to be substracted.
          
        Returns: 
            Dual: A dual number which contains the difference. 
        """
        try:
            return Dual(self.val - other.val, self.der - other.der)
        except AttributeError:
            return Dual(self.val - other, self.der)
        
    def __rsub__(self, other):
        """ 
        The function to substract a dual number from a float number. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (float): The float number to be substracted from.
          
        Returns: 
            Dual: A dual number which contains the difference. 
        """
        return Dual(other - self.val, -self.der)
        
    
    ## MULTIPLICATION OPERATIONS
    
    def __mul__(self, other):
        """ 
        The function to multiply two dual number and to multiply a dual number from the left. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (Dual / float): The dual/float number for multiplication.
          
        Returns: 
            Dual: A dual number which contains the product. 
        """
        try:
            # multiplication rule
            temp = self.val * other.der + self.der * other.val
            return Dual(self.val * other.val, temp)
        except AttributeError:
            return Dual(self.val * other, self.der * other)

    def __rmul__(self, other):
        """ 
        The function to multiply a dual number from the right. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (float): The float number for multiplication.
          
        Returns: 
            Dual: A dual number which contains the product. 
        """
        return Dual(self.val * other, self.der * other)
    
    
    ## DIVISION OPERATIONS
        
    def __truediv__(self, other):
        """ 
        The function to divide two dual number and to divide a dual number by a float number. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (Dual / float): The dual/float number for division.
          
        Returns: 
            Dual: A dual number which contains the quotient. 
        """
        try:
            # quotient rule
            temp = (self.der * other.val - self.val * other.der)
            print(self.der)
            print(other.der)
            return Dual(self.val/other.val, temp/other.val ** 2)
        except AttributeError:
            # divide by a constant
            return Dual(self.val/other, self.der/other)
        
    def __rtruediv__(self, other):
            """ 
            The function to divide a float by a dual number. 
  
            Parameters: 
                self (Dual): The current dual number.
                other (float): The float number for division.
          
            Returns: 
                Dual: A dual number which contains the quotient. 
            """
            return Dual(other/self.val, -other/self.val**2*self.der)   
    
    
    ## POWER OPERATIONS
    
    def __pow__(self, other):
        """ 
        The function to exponentiate a dual number by a float or dual number. 
  
        Parameters: 
            self (Dual): The current dual number.
            other (Dual / float): The dual/float number for exponentiation.
          
        Returns: 
            Dual: A dual number which contains the power. 
        """
        try:
            # da^u/dx = ln(a) a^u du/dx
            factor = self.val ** (other.val -1)
            sum_1 = other.val * self.der
            sum_2 = self.val * np.log(self.val) * other.der
            temp = factor * (sum_1 + sum_2)
            return Dual(self.val ** other.val, temp)
        except AttributeError:
            # du^n/dx = n * u^(n-1) * du/dx
            temp = other * self.val ** (other-1) * self.der
            return Dual(self.val ** other, temp)
        
    def __rpow__(self, other):
            """ 
            The function to exponentiate a float number by a dual number. 
  
            Parameters: 
                self (Dual): The current dual number.
                other (float): The float number for exponentiation.
          
            Returns: 
                Dual: A dual number which contains the power. 
            """
            # da^u/dx = ln(a) a^u du/dx
            temp = np.log(other) * other ** self.val * self.der
            return Dual(other ** self.val, temp)

In [3]:
#from Dual.dual import Dual
import inspect
class AutoDiff:
    def __init__(self, fn, ndim=1):
        self.fn = fn
        self.ndim = ndim
        sig = inspect.signature(self.fn)
        self.l = len(list(sig.parameters))         

    def get_der(self, val):
        """ Returns derivatives of the function evaluated at values given.
        
        INPUTS
        =======
        val : single number, a list of numbers, or a list of lists
        
        RETURNS
        =======
        Derivates in the same shape given
        
        EXAMPLE
        =======
        >>> a = AutoDiff(lambda x,y: 5*x + 4*y)
        a.get_der([[6.7, 4],[2,3],[4.5,6]])
        [[5, 4], [5, 4], [5, 4]]
        """
        ders = []
        if self.ndim >1:
            for i in range(self.ndim):
                def fxn(*args):
                    return self.fn(*args)[i]
                a = AutoDiff(fxn,ndim=1)
                a.l=self.l
                ders.append(a.get_der(val))
            return ders
        else:
            if self.l >= 2:

                #for list of lists, each list evaluated at different variables
                if any(isinstance(el, list) for el in val) is True:  
                    list_der = []
                    for p in val:
                        list_der.append(self.get_der(p))
                    return list_der
                elif self.l != len(val):
                    raise Exception('Function requires {} values that correspond to the multiple variables'.format(self.l))
                else:
                    #for a list of numbers, evaluated at different variables.
                    for i in range(self.l): 
                        new_val = val.copy()
                        new_val[i] = Dual(new_val[i])
                        v = self.fn(*new_val)                   
                        #Check if variable is in the function. (E.g., function paramaters are x, y and function is x.)
                        if type(v) is Dual:    
                            ders.append(self.fn(*new_val).der)
                        else:
                            ders.append(0)
                    return ders
            #for a list of numbers, evaluated at a single variable.i
            if (isinstance(val,list)): 
                for v in val:
                    a = Dual(v)
                    ders.append(self.fn(a).der)
                return ders
            else:
                a = Dual(val)
                return self.fn(a).der

In [4]:
def my_fn_2d(x, y):
    return [x**2 + y**2, x + 2+y]
def my_fn_1d(x,y):
    return x**2 + y**2
def fn1(x):
    return x**2

In [5]:
my_fn_1d(Dual(3),2)
boobie = AutoDiff(my_fn_1d,ndim=1)
boobie.get_der([1,2])

[2, 4]

In [6]:
boobie1 = AutoDiff(my_fn_2d,ndim=2)
np.array(boobie.get_der([1,2]))
boobie1.get_der([1,2])

[[2, 4], [1, 1]]

In [10]:
boobie1.get_der([1])

Exception: Function requires 2 values that correspond to the multiple variables

In [12]:
fn = lambda x, y: x**2

happy = AutoDiff(fn)
happy.get_der([1,2])

[2, 0]